
## Tabular Data Structures / DataFrames in Python for Engineers and Geoscientists 
### Michael Pyrcz, Associate Professor, University of Texas at Austin 

#### Contacts: [Twitter/@GeostatsGuy](https://twitter.com/geostatsguy) | [GitHub/GeostatsGuy](https://github.com/GeostatsGuy) | [www.michaelpyrcz.com](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446)

This is a tutorial for / demonstration of **Tabular Data Structures in Python**.  In Python, the common tool for dealing with Tabular Data Structures is the DataFrame from the pandas Python package. 

This tutorial includes the methods and operations that would commonly be required for Engineers and Geoscientists working with Tabular Data Structures for the purpose of:

1. Data Checking and Cleaning
2. Data Mining / Inferential Data Analysis
3. Data Analytics / Building Predictive Models with Geostatistics and Machine Learning

Learning to work with Pandas DataFrames is essential for dealing with tabular data (e.g. well data) in subsurface modeling workflows and for subsurface machine learning.

##### Tabular Data Structures

In Python we will commonly store our data in two formats, tables and arrays.  For sampled data with typically multiple features $1,\ldots,m$ over $1,\ldots,n$ samples we will work with tables.  For exhaustive maps and models usually representing a single feature on a regular grid over $1,\ldots,n_{i}$ for $i = 1,\ldots,n_{dim}$ we will work with arrays.

pandas package provides a convenient DataFrame object for working with data in a table and numpy package provides a convenient ndarray object for working with gridded data. In the following tutorial we will focus on DataFrames although we will utilize ndarrays a couple of times.  There is another section on Gridded Data Structures that focuses on ndarrays.

#### Project Goal

Learn the basics for working with Tabular Data Structures in Python.

#### Load the required libraries

The following code loads the required libraries.


In [1]:
import os                                                   # to set current working directory 
import numpy as np                                          # arrays and matrix math
import pandas as pd                                         # DataFrames

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.  



#### Set the working directory

I always like to do this so I don't lose files and to simplify subsequent read and writes (avoid including the full address each time).  Also, in this case make sure to place the required (see below) data file in this directory.  When we are done with this tutorial we will write our new dataset back to this directory.  

In [2]:
os.chdir("c:/PGE383")                                       # set the working directory

#### Loading Data 

Let's load the provided multivariate, spatial dataset.  '2D_MV_200wells.csv' is available at https://github.com/GeostatsGuy/GeoDataSets.  It is a comma delimited file with X and Y coordinates,facies 1 and 2 (1 is sandstone and 2 interbedded sand and mudstone), porosity (fraction), permeability (mDarcy) and acoustic impedance (kg/m2s*10^6). We load it with the pandas 'read_csv' function into a data frame we called 'df' and then preview it by printing a slice and by utilizing the 'head' DataFrame member function (with a nice and clean format, see below).

In [3]:
df = pd.read_csv("2D_MV_200wells.csv")                      # read a .csv file in as a DataFrame
#print(df.iloc[0:5,:])                                       # display first 4 samples in the table as a preview
df.head()                                                   # we could also use this command for a table preview 

,X,Y,facies_threshold_0.3,porosity,permeability,acoustic_impedance
0,565,1485,1,0.1184,6.170,2.009
1,2585,1185,1,0.1566,6.275,2.864
2,2065,2865,2,0.1920,92.297,3.524
3,3575,2655,1,0.1621,9.048,2.157
4,1835,35,1,0.1766,7.123,3.979


It is useful to review the summary statistics of our loaded DataFrame.  That can be accomplished with the 'describe' DataFrame member function.  We transpose to switch the axes for ease of visualization.

In [4]:
df.describe()

,X,Y,facies_threshold_0.3,porosity,permeability,acoustic_impedance
count,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000
mean,2053.400000,1876.15000,1.330000,0.149300,25.287462,3.000435
std,1113.524641,1137.58016,0.471393,0.032948,64.470135,0.592201
min,25.000000,35.00000,1.000000,0.050000,0.015820,2.009000
25%,1112.500000,920.00000,1.000000,0.132175,1.366750,2.483250
50%,2160.000000,1855.00000,1.000000,0.150150,4.825500,2.964500
75%,2915.000000,2782.50000,2.000000,0.174200,14.597000,3.527000
max,3955.000000,3995.00000,2.000000,0.223200,463.641000,3.984000


Let's rename the facies, permeability and acoustic impedance for convenience.

In [5]:
df = df.rename(columns={'facies_threshold_0.3': 'facies','permeability':'perm','acoustic_impedance':'ai'}) # rename columns of the 
df.head()

,X,Y,facies,porosity,perm,ai
0,565,1485,1,0.1184,6.170,2.009
1,2585,1185,1,0.1566,6.275,2.864
2,2065,2865,2,0.1920,92.297,3.524
3,3575,2655,1,0.1621,9.048,2.157
4,1835,35,1,0.1766,7.123,3.979


It is straightforward to extract subsets from a DataFrame to make a new DataFrame.  This is useful for cleaning up data by removing features that are no longer of interest.  If the samples are in random order then the first $n_{s}$ samples are a random sample of size $n_{s}$.  Below we make a new DataFrame, 'df_subset', with the rows 0 to 4 and columns 2 to 6 and another new DataFrame, 'df_no_location', with the X and Y coordinates removed.

In [6]:
df_subset = df.iloc[0:5,2:7]                                # make a new dataframe with just the first 4 samples and no X,Y
#df_subset.head()
print(df_subset)

   facies  porosity    perm     ai
0       1    0.1184   6.170  2.009
1       1    0.1566   6.275  2.864
2       2    0.1920  92.297  3.524
3       1    0.1621   9.048  2.157
4       1    0.1766   7.123  3.979


It is also easy to add a column to our data frame.  Note, we assume that the array is in the same order as the DataFrame.  This could be an issue if any rows were removed form either before adding etc.  To demonstrate we make a 1D numpy array of zeros using the 'zeros' function and add it to our DataFrame with the feature name indicated as 'zero'.

In [7]:
zeros = np.zeros(200)                                       # make a array of zeros
df['zero'] = pd.Series(zeros)                               # add the array to our DataFrame
df.head()

,X,Y,facies,porosity,perm,ai,zero
0,565,1485,1,0.1184,6.170,2.009,0.0
1,2585,1185,1,0.1566,6.275,2.864,0.0
2,2065,2865,2,0.1920,92.297,3.524,0.0
3,3575,2655,1,0.1621,9.048,2.157,0.0
4,1835,35,1,0.1766,7.123,3.979,0.0


We can also remove unwanted columns without having to subset the DataFrame.  We do this with the 'drop' member function of the DataFrame object. We just have the give the column name and by indicating axis=1 we specify to drop a column instead of a row.

In [8]:
df = df.drop('zero',axis=1)                                      # remove the zero column
df.head()

,X,Y,facies,porosity,perm,ai
0,565,1485,1,0.1184,6.170,2.009
1,2585,1185,1,0.1566,6.275,2.864
2,2065,2865,2,0.1920,92.297,3.524
3,3575,2655,1,0.1621,9.048,2.157
4,1835,35,1,0.1766,7.123,3.979


We may want to make new features by using mathematical operators applied to existing features.  For example we can make a porosity feature that is in percentage instead of fraction (called 'porosity100') or a ratio of permeability divided by porosity (called 'permpor') may be useful for subsequent calculations such as the Lorenz Coefficient.  

In [9]:
df['porosity100'] = df['porosity']*100                      # add a new column with porosity in percentage
df['permpor'] = df['perm']/df['porosity']           # add a new feature with ratio of perm / por 
df.head()

,X,Y,facies,porosity,perm,ai,porosity100,permpor
0,565,1485,1,0.1184,6.170,2.009,11.84,52.111486
1,2585,1185,1,0.1566,6.275,2.864,15.66,40.070243
2,2065,2865,2,0.1920,92.297,3.524,19.20,480.713542
3,3575,2655,1,0.1621,9.048,2.157,16.21,55.817397
4,1835,35,1,0.1766,7.123,3.979,17.66,40.334088


We could also use conditional statements when assigning values to a new feature.  For example, we could have a categorical porosity measure for high and low porosity, called 'tporosity'.

In [10]:
df['tporosity'] = np.where(df['porosity']>=0.12, 'high', 'low') # conditional statement assign a new feature
df.head()

,X,Y,facies,porosity,perm,ai,porosity100,permpor,tporosity
0,565,1485,1,0.1184,6.170,2.009,11.84,52.111486,low
1,2585,1185,1,0.1566,6.275,2.864,15.66,40.070243,high
2,2065,2865,2,0.1920,92.297,3.524,19.20,480.713542,high
3,3575,2655,1,0.1621,9.048,2.157,16.21,55.817397,high
4,1835,35,1,0.1766,7.123,3.979,17.66,40.334088,high


Here's an example where we use a conditional statement to assign a very low permeability value (0.0001 mD) for all porosity values below a threshold. Of course, this is for demonstration, in practice a much lower porosity threshold would likely be applied.  

In [11]:
df['perm_cutoff'] = np.where(df['porosity']>=0.12, df['perm'],0.0001) # conditional statement assign a new feature
df.head()

,X,Y,facies,porosity,perm,ai,porosity100,permpor,tporosity,perm_cutoff
0,565,1485,1,0.1184,6.170,2.009,11.84,52.111486,low,0.0001
1,2585,1185,1,0.1566,6.275,2.864,15.66,40.070243,high,6.2750
2,2065,2865,2,0.1920,92.297,3.524,19.20,480.713542,high,92.2970
3,3575,2655,1,0.1621,9.048,2.157,16.21,55.817397,high,9.0480
4,1835,35,1,0.1766,7.123,3.979,17.66,40.334088,high,7.1230


What about missing or invalid values?  Let's assign a single porosity value to NaN, 'not a number', indicating a missing or eroneous value.  We will then check for the number of NaN values in our DataFrame.  Then we can search for and display the sample with the NaN porosity value.

In [12]:
df.set_value(1,'porosity',np.NaN)                           # let's give ourselves a NaN / missing value in our table
print('Number of null values in our DataFrame = ', str(df.isnull().sum().sum()))                                     # let's check for missing values in our table
nan_rows = df[df['porosity'].isnull()]                      # find the row with missing values
print(nan_rows)

Number of null values in our DataFrame =  1
      X     Y  facies  porosity   perm     ai  porosity100    permpor  \
1  2585  1185       1       NaN  6.275  2.864        15.66  40.070243   

  tporosity  perm_cutoff  
1      high        6.275  


C:\Users\pm27995\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


We can see that sample 1 has a NaN porosity value.  Now we may choose to remove the sample with the NaN.  The 'dropna' DataFrame member function will remove all samples with NaN entries from the entire DataFrame.  By visualizing the index at the left of the DataFrame preview you can confirm that sample 1 is removed (it jumps from 0 to 2).

In [13]:
df = df.dropna()                                            # drop any rows (samples) with atleast one missing value        
df.head()

,X,Y,facies,porosity,perm,ai,porosity100,permpor,tporosity,perm_cutoff
0,565,1485,1,0.1184,6.170,2.009,11.84,52.111486,low,0.0001
2,2065,2865,2,0.1920,92.297,3.524,19.20,480.713542,high,92.2970
3,3575,2655,1,0.1621,9.048,2.157,16.21,55.817397,high,9.0480
4,1835,35,1,0.1766,7.123,3.979,17.66,40.334088,high,7.1230
5,3375,2525,1,0.1239,1.468,2.337,12.39,11.848265,high,1.4680


You may want to rename the feature names, column titles.  This is accomplished with the 'rename' DataFrame member function.

One could extract samples into a new DataFrame with multiple criteria.  This is shown below.

In [14]:
df_extract = df.loc[(df['porosity'] > 0.12) & (df['perm'] > 10.0)] # extract with multiple conditions to a new table
df_extract.head()

,X,Y,facies,porosity,perm,ai,porosity100,permpor,tporosity,perm_cutoff
2,2065,2865,2,0.1920,92.297,3.524,19.20,480.713542,high,92.297
6,2295,1325,1,0.1790,31.933,3.491,17.90,178.396648,high,31.933
7,3715,3045,2,0.1914,116.781,2.187,19.14,610.141066,high,116.781
13,545,3765,1,0.1817,14.311,3.045,18.17,78.761695,high,14.311
15,1385,2415,2,0.1774,22.578,2.711,17.74,127.271702,high,22.578


It is also simple to build a brandnew DataFrame from a set of 1D arrays.  Note, they must have the same size and be sorted consistently.  We will extract porosity and perm as arrays (if you remove '.values' they are extracted as Series, an array retaining the feature name).  We then use the pandas DataFrame command to make a new DataFrame with each 1D array and the column names specified as 'porosity' and 'permeabilty'.

In [22]:
por = df['porosity'].values                                 # extract porosity column as vector
perm = df['perm'].values                                    # extract permeability column as vector
df_new = pd.DataFrame({'porosity': por, 'permeability': perm}) # make a new DataFrame from the vectors
df_new.head()

,porosity,permeability
0,0.1184,6.170
1,0.1920,92.297
2,0.1621,9.048
3,0.1766,7.123
4,0.1239,1.468


We can reach in and retrieve the actual raw information in the DataFrame including the column names and actual values as an numpy array.  We can't edit them like this, but we can access and use this information.  This included 'index' with information about the index (i.e. index from start to stop with step), 'columns' with the names of the features and 'values' with the data table entries as a 2D array.   

In [23]:
print(df.index)                                             # get information about the index
print(df.columns)                                           # get the list of feature names
print(df.values)                                            # get the 2D array with all the table data

Int64Index([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            190, 191, 192, 193, 194, 195, 196, 197, 198, 199],
           dtype='int64', length=199)
Index(['X', 'Y', 'facies', 'porosity', 'perm', 'ai', 'porosity100', 'permpor',
       'tporosity', 'perm_cutoff'],
      dtype='object')
[[565 1485 1 ... 52.111486486486484 'low' 0.0001]
 [2065 2865 2 ... 480.71354166666674 'high' 92.29700000000001]
 [3575 2655 1 ... 55.81739666872301 'high' 9.048]
 ...
 [375 1705 1 ... 18.198334595003786 'high' 2.404]
 [3795 535 1 ... 0.25968483256730135 'low' 0.0001]
 [3455 1645 1 ... 6.578073089700997 'high' 0.99]]


Another method for getting a list of the DataFrame feature names.

In [24]:
list(df)                                                    # get a list with the feature names

['X',
 'Y',
 'facies',
 'porosity',
 'perm',
 'ai',
 'porosity100',
 'permpor',
 'tporosity',
 'perm_cutoff']

Let's inteact with the DataFrame more surgically, one feature and sample at a time.  Here we retrieve the 4th column feature name and the porosity value for sample #1.  

In [25]:
col2_name = df.columns[3]                                   # get the name of the 4th feature (porosity)
print(col2_name)                                          
por1 = df.values[1,3]                                       # get the value for sample 1 of the 4th feature (porosity)
print('Porosity value for sample number 1 is ' + str(por1) + '.') 

porosity
Porosity value for sample number 1 is 0.192.


We can also manually change values.  This is done with the 'iloc' pandas DataFrame member function for integer-location based selection.  We look up the porosity value for sample 1 and then we use the 'set_value' DataFrame member function to change the value to 0.1000.  

In [26]:
por = df.iloc[2]['porosity']                               # get the value for sample 1 of the porosity feature
print('The value of porosity for sample 2 is ' + str(por) + '.')
df.set_value(2,'porosity',0.1000)                          # set the value for sample 1 of the porosity feature
print('The value of porosity for sample 2 is now 0.1000.')
df.head()

The value of porosity for sample 2 is 0.1621.
The value of porosity for sample 2 is now 0.1000.


C:\Users\pm27995\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


,X,Y,facies,porosity,perm,ai,porosity100,permpor,tporosity,perm_cutoff
0,565,1485,1,0.1184,6.170,2.009,11.84,52.111486,low,0.0001
2,2065,2865,2,0.1000,92.297,3.524,19.20,480.713542,high,92.2970
3,3575,2655,1,0.1621,9.048,2.157,16.21,55.817397,high,9.0480
4,1835,35,1,0.1766,7.123,3.979,17.66,40.334088,high,7.1230
5,3375,2525,1,0.1239,1.468,2.337,12.39,11.848265,high,1.4680


It may be useful to write the DataFrame out for storage or curation and / or to be utilize with another platform (even R or Excel!).  It is easy to write the DataFrame back to a comma delimited file.  We have the 'to_csv' DataFrame member function to accomplish this.  The file will write to the working directory (another reason we set that at the beginning).  Go to that folder and open this new file with TextPad, Excel or any other program that opens .txt files to check it out.

In [ ]:
df.to_csv("2D_MV_200wells_out.csv")                      # write out the df DataFrame to a comma delimited file 

#### More Exercises

There are so many more exercises and tests that one could attempt to gain experience with the pandas package, DataFrames objects in Python. I'll end here for brevity, but I invite you to continue. Check out the docs at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html.  I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin
On twitter I'm the @GeostatsGuy.
